# Cálculos Índice

Aquí hare todo el plotting the los precios del indice, aunque este a medias creo la base para solo tener que cambiar el archivo del cual saco la informacion y apa ya esta. Dejandolo Listo para el siguiente paso.
Seleccionar los parámetros de los precios, crear la fórmula para el cálculo de los pesos y crear la función que calcule el índice.

In [ ]:
import pandas as pd
import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

path = '/Users/ruben/Documents/MASTER/TFM/data/'
a = pd.read_csv(path + 'draft1.csv')
a1=a.drop(['Unnamed: 0'], axis=1)
a1.columns

In [ ]:
a1.sort_values(by = 'Company Market Cap')[['Name', 'TRBC Industry Group Name']]

In [ ]:
c, err = ek.get_data(a1.RICS.to_list(),['TR.CLOSEPRICE.date','TR.CLOSEPRICE', 'TR.CompanyMarketCap'],
            {'SDate':'2018-04-20', 'Curn':'EUR', 'EDate':'2023-04-20', 'Adjusted':'1'}) # En euros para evitar problemas de divisa y ajustado para introducir eventos de los stocks
print('total: '+c.__len__(), 'Dates: '+ c['Date'].unique())
c # What if weekly to avoid data loss? or Drop Data that has less than 45 instruments?


# AQUÍ METEMOS EL FLOAT-RATE POR EMPRESA

In [ ]:
c = pd.DataFrame(c, columns = ['Instrument', 'Date', 'Close Price','rent_diaria', 'Company Market Cap', 'weight'])
total_cap = []

for i in c['Date'].unique(): # Calculo de la capitalización total de cada fecha
   total_cap.append(sum(c[c['Date']==i]['Company Market Cap']))

full = pd.DataFrame( columns = ['Date', 'total_cap'])
full['total_cap']= total_cap
full['Date'] = c['Date'].unique()

for i, v in zip(c['Date'], range(len(c))): # Calculo de los pesos de cada stock en cada fecha
    c['weight'].iloc[v]=c[c['Date']==i]['Company Market Cap'][v]/sum(full[full['Date']==i]['total_cap'])

index = pd.MultiIndex.from_frame(c[['Date','weight']])
c_index = c.set_index(index, inplace= False)
c_index.sort_index(level=['Date', 'weight'], inplace=True, ascending=[True, False])
c_index

In [ ]:
index = pd.MultiIndex.from_frame(c[['Instrument', 'Date']])
b_index = c.set_index(index, inplace= False)
b_index.sort_index(level=['Instrument', 'Date'], inplace=True, ascending=[True, True])
b_index = pd.DataFrame(b_index ,columns = ['Instrument', 'Date', 'Close Price', 'Company Market Cap', 'weight', 'rent_diaria'])

b=[]
for i in b_index['Instrument'].unique(): # rentabilidad diaria y aportada al indice
    b.append(b_index[b_index['Instrument']==i]['Close Price'].pct_change())

for i, v in zip(b_index['Instrument'].unique(), range(len(b_index['Instrument'].unique()))):
    b_index.loc[i, 'rent_diaria']=b[v].values

b_index['rent_index'] = b_index['weight']*b_index['rent_diaria']
b_index

In [ ]:
bnew =b_index.reset_index(drop=True, inplace=False)
index = pd.MultiIndex.from_frame(b_index[['Date','Instrument']])
bnew.set_index(index, inplace=True)
bnew.sort_index(level=['Date', 'Instrument'], inplace=True, ascending=[True, True])

full =  pd.DataFrame(full, columns = ['Date', 'total_cap', 'tot_rent'])
for i, v in zip(bnew['Date'].unique(), range(len(bnew['Date'].unique()))):
    full['tot_rent'].iloc[v]=bnew.loc[i]['rent_index'].sum()
full # rentabilidad y capitalización diaria del índice

In [ ]:
full =  pd.DataFrame(full, columns = ['Date', 'total_cap', 'tot_rent', 'net value'])
full['net value'].iloc[0]=1000 # Valor inicial del índice
for i in range(len(full)): # Valor neto del índice
    if IndexError:
        pass
    else:
        full['net value'].iloc[i+1]=full['net value'].iloc[i]*(1+full['tot_rent'].iloc[i+1])

b_index.to_csv(path + 'b_index.csv')
full.to_csv(path + 'full.csv')

full

# Segunda parte, pyfolio

In [ ]:
full[full['net value']==full['net value'].min()] # Mínimo valor del índice -> Covid jaja 

In [ ]:
full['Date'] = pd.to_datetime(full['Date'])
full['Date'] = full['Date'].dt.strftime('%Y-%m-%d')
full['Date'] = pd.to_datetime(full['Date'])


In [ ]:
import eikon as ek

ek.set_app_key('f5c3af98ce3e4070aba80b9bc465cd82b9624cfc')

import pandas as pd
import pyfolio as pf
import numpy as np
# Silence warnings
import warnings
warnings.filterwarnings('ignore')

path = '/Users/ruben/Documents/MASTER/TFM/data/'
#full = pd.read_csv(path+'full.csv', parse_dates=['Date'])
#bac = full.set_index('Date')
#bac.drop('Unnamed: 0', axis=1, inplace=True)
#bac['net value'].resample('A').ffill().pct_change() # Rentabilidad Anual / Mensual / Semanal del índice

In [ ]:
# rentabilidades diarias de los índices mundiales -> ready en rets.csv (con rets adj)
rics = ['.FTSE', '.STOXX50E']
bench_rets = ek.get_timeseries(rics, fields = 'CLOSE',start_date='2018-04-20', end_date='2023-04-20', interval='daily')
a = pd.DataFrame(bench_rets.pct_change())
test = bac.merge(a, on = 'Date', how='left')
tics = ['.SX5EESG','.FT4GDGLOB100']
bench_rets_1 = ek.get_timeseries(tics, fields = 'CLOSE',start_date='2018-04-20', end_date='2023-04-20', interval='daily')
b = pd.DataFrame(bench_rets_1.pct_change())
test = test.merge(b, on = 'Date', how='left')
dics = ['.MIWO00000PUS', '.NDX']
bench_rets_2 = ek.get_timeseries(dics, fields = 'CLOSE',start_date='2018-04-20', end_date='2023-04-20', interval='daily')
c = pd.DataFrame(bench_rets_2.pct_change())
test = test.merge(c, on = 'Date', how='left')
bics = ['.SXWESGP']
bench_rets_3 = ek.get_timeseries(bics, fields = 'CLOSE',start_date='2018-04-20', end_date='2023-04-20', interval='daily')
d = pd.DataFrame(bench_rets_3.pct_change())
test = test.merge(d, on = 'Date', how='left')
test.rename(columns = {'CLOSE' : '.SXWESGP'}, inplace=True)
rets = test[['tot_rent','.FT4GDGLOB100','.STOXX50E','.FTSE', '.STOXX50','.SXWESGP', '.MIWO00000PUS', '.NDX']]
rets = rets.ffill(axis  = 1)

In [ ]:
rets.drop(columns = ['Unnamed: 0'], inplace=True)


In [ ]:
rets.index

In [ ]:
#rets with less than 40 comp per day 
rets_null = pd.read_csv(path+'index_adj_null.csv', parse_dates=['Date'])
rets_null.drop('Unnamed: 0', axis=1, inplace=True)
rets_null['Date']= rets_null['Date'].dt.strftime('%Y-%m-%d')
rets_null.rename(columns = {'tot_rent_adj' : 'tot_rent_adj_null'}, inplace=True)

rets_null['Date'] = pd.to_datetime(rets_null['Date'], yearfirst=True)
rets_null.set_index('Date', inplace=True)
rets_null.index

In [ ]:
rets_null['Date'] = pd.to_datetime(rets_null['Date'], yearfirst=True)
rets_null.set_index('Date', inplace=True)
rets_null.index


In [ ]:
#sector mapping
beta = pd.read_csv(path + 'gold.csv', parse_dates=['Date'])

sector_map = beta[['Instrument', 'TRBC Economic Sector Name']].set_index('Instrument').to_dict()
sector_map['TRBC Economic Sector Name']
pf.timeseries.annual_return(rets['tot_rent_adj'], period='daily')
pf.timeseries.forecast_cone_bootstrap(rets['tot_rent_adj'], 30, cone_std=(1.0, 1.5, 2.0), starting_value=1, num_samples=1000, random_seed=None)

In [ ]:
# Tear Sheet w Bench and updated rets
rets = pd.read_csv(path+'rets.csv', parse_dates=['Date'])
rets.set_index('Date', inplace=True)
pf.create_full_tear_sheet(rets_null['tot_rent_adj_null'], benchmark_rets=rets['.STOXX50']) # Análisis de la rentabilidad del índice

In [ ]:
# Cumulative Returns Chart w Benchmark
import matplotlib.pyplot as plt

pf.create_interesting_times_tear_sheet(rets['tot_rent_adj'], benchmark_rets=rets['.STOXX50'])

In [ ]:
# Stats for the Index in table format
stats = pf.timeseries.perf_stats(rets['tot_rent_adj'])
pd.DataFrame(stats, columns=['Values'])

In [ ]:
# Carbon emissions data 

cc_index = pd.read_csv(path+'weight_ind.csv', parse_dates=['Date'])
c, err = ek.get_data(cc_index.Instrument.unique().tolist(),['TR.AnalyticGHGEmissionsScope1and2and3(Period=FY0).date', 'TR.AnalyticGHGEmissionsScope1and2and3(Period=FY0)'])
c

# Intensidad de emisiones del Índice -> usar esto para los demás índices y comparar la intensidad de emisiones entre todos 
#cc_index['Date'] = cc_index['Date'].dt.strftime('%Y-%m-%d')
index = pd.MultiIndex.from_frame(cc_index[['Instrument', 'Date']])
b_index = cc_index.set_index(index) # cc_index -> not indexed // b_index -> indexed
b_index.drop(columns = ['Instrument', 'Date','Unnamed: 0'], inplace=True)

means  = pd.DataFrame(data = b_index.index.get_level_values(0).unique(), columns = ['Instrument', 'weight_adj'])

for i, v in zip(means['Instrument'], range(len(means['Instrument']))): # avg means of weights for 2022
    means.iloc[v]['weight_adj'] = b_index.loc[i].loc['2022']['weight_adj'].mean()

boi = c.merge(means, on = 'Instrument', how='left')
boi['weighted emissions in MILL'] = boi['GHG Emissions Scope 1,2,3 to Revenue USD in million']*boi['weight_adj']
boi.sum() # suma de las emisiones a revenue ponderadas por peso del índice en millones de USD

In [ ]:
# orden de los stocks por peso en el índice
means.sort_values(by = 'weight_adj', ascending=False).head(10)

In [ ]:
# Index lists w returns
index_list = ['.FTSE', '.STOXX50', '.STOXX50E', '.FT4GDGLOB100', '.SXWESGP', '.MIWO00000PUS', '.NDX', '.IBEX35']
d, err = ek.get_data(index_list, ['TR.IndexName','TR.IndexMktCapVendor','TR.IndexTotalReturn1Yr','TR.IndexTotalReturn3Yr','TR.IndexTotalReturn5Yr'])
d

In [ ]:
# Index constituents and weights for indices (pesos no encontrados hay que mirar más -> comentarlo en PPT)
index_con = ['.SXWESGP','.FTSE', '.STOXX50', '.STOXX50E', '.NDX', '.FT4GDGLOB100']
teta=pd.DataFrame(columns = ['Index'])
for i in index_con:
    ab, err = ek.get_data(i, ['TR.IndexConstituentRIC','TR.IndexConstituentWeightPercent'])
    teta.insert(0, str(i) ,ab['Constituent RIC'])
    teta.insert(0, str(i+' weight') ,ab['Weight percent'])
teta.drop('Index', axis=1, inplace=True)
teta

In [ ]:
# Alineación con la disminución del 7% de emisiones anuales para cumplir con el acuerdo de Paris de Índices y Mío
import numpy as np
index_con = ['.SXWESGP','.FTSE', '.STOXX50', '.STOXX50E', '.NDX']
l = []
l.extend(range(6))
pcts = pd.DataFrame(columns = ['empty','indice', 'pct alineado'])
pcts['empty']=l

for i, k in zip(index_con, range(6)):
    ba, err = ek.get_data(teta[i].dropna().to_list(), ['TR.AnalyticGHGEmissionsIntensityScope1and2and3ParisAligned(Period=FY0)'])
    ba = ba.groupby('GHG Emissions Intensity Scope 1,2,3 Paris Agreement Aligned').count()
    pcts['indice'].iloc[k]=i
    pcts['pct alineado'].iloc[k]=np.round((ba.loc['True']['Instrument']/ba['Instrument'].sum())*100, 2)

ba, err = ek.get_data(cc_index['Instrument'].unique().tolist(), ['TR.AnalyticGHGEmissionsIntensityScope1and2and3ParisAligned(Period=FY0)'])
ba = ba.groupby('GHG Emissions Intensity Scope 1,2,3 Paris Agreement Aligned').count()
pcts['indice'].iloc[5]='own'
pcts['pct alineado'].iloc[5]=np.round((ba.loc['True']['Instrument']/ba['Instrument'].sum())*100, 2)

pcts.drop('empty', axis = 1, inplace = True) # % de empresas alineadas con el acuerdo de Paris por índice
pcts

In [ ]:
# Resumen de los datos base del índice
a1 = pd.read_csv(path + 'draft1.csv')
index2 = pd.MultiIndex.from_frame(a1[['TRBC Economic Sector Name', 'TRBC Industry Group Name']])
b2 = a1.set_index(index2, inplace=False).rename(columns = {'RICS':'Instrument'})
b2.sort_index(level=['TRBC Economic Sector Name', 'TRBC Industry Group Name'], inplace=True, ascending=[True, True])
b3 = b2.merge(boi, on = 'Instrument', how='left')
main = b3[['Name', 'Instrument','weight_adj', 'TRBC Economic Sector Name', 'Company Market Cap','GHG Emissions Scope 1,2,3 to Revenue USD in million']]
main.sort_values(by = 'weight_adj', ascending=False, inplace=True)
main
b3.to_csv(path + 'b3.csv') # Datos completos del Índice con pesos media de 2022
main.to_csv(path + 'main.csv') # Datos base del índice con pesos media de 2022

In [ ]:
# Peso y Emisiones de cada sector en el índice

b3 = pd.read_csv(path + 'b3.csv')
indus_peso = pd.DataFrame(b3.groupby(['TRBC Economic Sector Name'])['weight_adj'].sum().sort_values(ascending=False), columns = ['weight_adj'])
indus_emi = pd.DataFrame(b3.groupby(['TRBC Economic Sector Name'])['GHG Emissions Scope 1,2,3 to Revenue USD in million'].sum())
display(indus_peso,indus_emi) 

In [ ]:
# Peso por sector
import seaborn as sns 
sns.set_theme(style="darkgrid")
sns.set_context("poster", font_scale=0.8)
sns.barplot(x=indus_peso['weight_adj'], y=indus_peso.index, data=indus_peso, palette='Reds_d')
sns.despine()

In [ ]:
# Tratamos datos para calcular la rentabilidad por sector 
cols = b3[['Instrument', 'TRBC Economic Sector Name', 'TRBC Industry Group Name']]
b_index['rent_index_adj'] = b_index['weight_adj']*b_index['rent_diaria']
rent_sector = b_index.merge(cols, on = 'Instrument', how='left')
rent_sector.set_index(index, inplace=True)

In [ ]:

index = pd.MultiIndex.from_frame(cc_index[['Instrument', 'Date']])
b_index = cc_index.set_index(index) # cc_index -> not indexed // b_index -> indexed
b_index['rent_index_adj'] = b_index['weight_adj']*b_index['rent_diaria']
b_index.drop(columns = ['Unnamed: 0'], inplace=True)
b_index

In [ ]:
# Cálculo real de las rentabilidades diarias por sector -> Podemos hacer Stacked Area Chart con esto
cols = b3[['Instrument', 'TRBC Economic Sector Name', 'TRBC Industry Group Name']]

rent_sector = b_index.merge(cols, on = 'Instrument', how='left')
rent_sector.set_index(index, inplace=True)
rent_sector.drop(columns = ['Instrument'], inplace=True)
rent_sector.reset_index(inplace=True)

rent_sector['Date'] = pd.to_datetime(rent_sector['Date'])
rent_sector['Date'] = rent_sector['Date'].dt.strftime('%Y-%m-%d')
index2 = pd.MultiIndex.from_frame(rent_sector[['TRBC Economic Sector Name','Date']])
rent_sector.set_index(index2, inplace=True)
rent_sector.sort_index(level=['TRBC Economic Sector Name','Date'], inplace=True, ascending=[True, True])
rent_sector.drop(columns = ['TRBC Economic Sector Name', 'Date'], inplace=True)

dates =  rent_sector.index.get_level_values(1).unique()
dates1 = rent_sector.index.get_level_values(1).unique()
for i in range (9): 
    dates = dates.append(dates1)
alpha = pd.DataFrame(dates, columns = ['Date','TRBC Economic Sector Name', 'tot_rent_adj', 'tot_cap_adj'])
loc = 0
for i in rent_sector.index.get_level_values(0).unique():
    for k in range(len(alpha['Date'].unique())):
        alpha['TRBC Economic Sector Name'].iloc[loc]=i
        loc = loc + 1
loc = 0
index = pd.MultiIndex.from_frame(alpha[['TRBC Economic Sector Name','Date']])
alpha.set_index(index, inplace=True)
beta = alpha.drop(columns = ['TRBC Economic Sector Name', 'Date'])
beta.sort_index(level=['TRBC Economic Sector Name','Date'], inplace=True, ascending=[True, True])
b = []
for k in rent_sector.index.get_level_values(0).unique():
    temp = rent_sector.loc[k]
    for i, v in zip(alpha.index.get_level_values(1).unique(), range(len(beta))):
        try:
            beta['tot_rent_adj'].iloc[loc]=temp.loc[i]['rent_index_adj'].sum()
            b.append(temp.loc[i]['rent_index_adj'].sum()) # Este es el que vale lo de arriba es basura
        except KeyError:
            beta['tot_rent_adj'].iloc[loc]= np.nan
            
        loc = loc + 1
casi = pd.DataFrame(b, columns = ['rent_sector_adj'], index = rent_sector.index.unique())
casi
#gamma # rentabilidad y capitalización diaria del índice

In [ ]:
# Renatbilidad acumulada por Sector y por Compañía de manera ajena al índice
d=[]
e = []
c = []
for i in casi.index.get_level_values(0).unique():
    e.append(i)
    c = (1 + casi.loc[i].rent_sector_adj).cumprod() - 1
    d.append(c)

best = pd.DataFrame(d, index = e)
rets_sec = pd.DataFrame(best['2023-04-20'])
rets_sec.sort_values(by = '2023-04-20', ascending=False, inplace=True)


f=[]
g = []
c =[]
for i in b_index.index.get_level_values(0).unique():
    g.append(i)
    c = (1 + b_index.loc[i].rent_index_adj).cumprod() - 1
    f.append(c[-1])
cata = pd.DataFrame(f, index = g)
rets_comp = pd.DataFrame(cata)
rets_comp.rename(columns = {0 : '2023-04-20'}, inplace=True)
rets_comp.sort_values(by = '2023-04-20', ascending=False, inplace=True)
display(rets_sec, rets_comp.head(10))

In [ ]:

#rets_comp.reset_index(inplace=True)
#rets_comp.rename(columns = {'2023-04-20' : 'Rentabilidad Acumulada', 'index': 'Instrument'}, inplace=True)
#rets_comp.drop(columns = ['level_0'], inplace=True)
rets_comp.merge(beta[['Name','Instrument']], on = 'Instrument', how='left').set_index('Name').drop(columns='Instrument').head(10)

In [ ]:
# PCT de tamaños de empresas
vol_cap = b3.groupby('Market Cap Category').count()['ISIN']
l = vol_cap[0]/vol_cap.sum()
m = vol_cap[1]/vol_cap.sum()
s = vol_cap[2]/vol_cap.sum()

print ('Large Cap: '+str(l), '%', 'Mid Cap: '+str(m), '%' ,'Small Cap: '+str(s), '%')

In [ ]:
# Top 10 basado en last pesos índice -> antes y ahora (2023)
testo = b_index.reset_index(drop= True).set_index(['Date'])
a = testo.loc['2023-04-20'][['Instrument','weight_adj']].rename(columns = {'weight_adj':'weight_adj_2023'})
b = testo.loc['2023-04-20'][['Instrument','Company Market Cap']].rename(columns = {'Company Market Cap':'mkt_cap_2023_EUR'})
b3 = b3.merge(b, on = 'Instrument', how='left')
beta = b3.merge(a, on = 'Instrument', how='left')
print(beta[['Name', 'weight_adj_2023', 'TRBC Economic Sector Name']].sort_values(by = 'weight_adj_2023', ascending=False).set_index('Name').head(10).sum())

testo = b_index.reset_index(drop= True).set_index(['Date'])
a = testo.loc['2023-04-20'][['Instrument','weight']].rename(columns = {'weight':'weight_2023'})
beta = beta.merge(a, on = 'Instrument', how='left')
print(beta[['Name', 'weight_2023', 'TRBC Economic Sector Name']].sort_values(by = 'weight_2023', ascending=False).set_index('Name').head(10).sum())
display(beta[['Name', 'weight_adj_2023', 'TRBC Economic Sector Name']].sort_values(by = 'weight_adj_2023', ascending=False).set_index('Name').head(10),
        beta[['Name', 'weight_2023', 'TRBC Economic Sector Name']].sort_values(by = 'weight_2023', ascending=False).set_index('Name').head(10))

In [ ]:
beta.drop(columns = ['Unnamed: 0', 'mkt_cap_2023_EUR_y' ], inplace = True)
beta.rename(columns = {'mkt_cap_2023_EUR_x':'mkt_cap_2023_EUR'}, inplace=True)
beta.to_csv(path + 'gold.csv') # Datos completos del Índice

In [ ]:
beta = pd.read_csv(path + 'gold.csv', parse_dates=['Date'])

sector_map = beta[['Instrument', 'TRBC Economic Sector Name']].set_index('Instrument').to_dict()


In [ ]:
sector_map

In [ ]:
# Weights by Sector for 2023 New vs Old
display(beta.groupby('TRBC Economic Sector Name')[['weight_adj_2023', 'TRBC Economic Sector Name']].sum().sort_values(by='weight_adj_2023', ascending=False).rename(columns = {'weight_adj_2023':'weights'}, inplace = False),
        beta.groupby('TRBC Economic Sector Name')[['weight_2023', 'TRBC Economic Sector Name']].sum().sort_values(by='weight_2023', ascending=False))

In [ ]:
# Visualización de los pesos de cada sector en el índice
import seaborn as sns 
import numpy as np

beta['log_cap_2023'] = np.log(beta['mkt_cap_2023_EUR'])

# drawing the plot
sns.scatterplot(data = beta, x= 'log_cap_2023' , y = 'weight_adj_2023', hue = 'TRBC Economic Sector Name')
sns.set(rc={'figure.figsize':(20,10)})


In [ ]:
countries = ek.get_data(b3.Instrument.unique().tolist(),['TR.HeadquartersCountry','TR.ExchangeCountry','CURRENCY'])
countries

In [ ]:
plot_cont = countries[0].groupby('Country of Headquarters').count().sort_values(by = 'Instrument', ascending=False)
sns.set_theme(style="darkgrid")
sns.set_context("poster", font_scale=0.8)
sns.barplot(x=plot_cont.Instrument, y=plot_cont.index, data=plot_cont, palette='Reds_d')
sns.despine()

In [ ]:
countries

In [ ]:
mask = c.groupby(c['Date'])['Instrument'].count() == 45
mask[mask==False]

In [ ]:
ab = c[c['Date']=='2018-04-27T00:00:00Z']['Instrument'].to_list()
a1[a1['RICS'].isin(ab)]